## Metoda eliminacji Gaussa

Rozwiązanie układu równań liniowych metodą eliminacji Gaussa polega na przekształceniu macierzy współczynników układu równań na macierz trójkątną górną, a następnie na rozwiązaniu układu równań za pomocą metody podstawiania wstecz.

Jest to jedna z najbardziej podstawowych metod rozwiązywania układów równań liniowych, wychodząca poza sytuacje gdzie liczba niewiadomych jest różna od liczby równań.

In [ ]:
from sympy import Matrix, symbols, Eq, latex
from IPython.display import display, Markdown
import sympy as sp

class SymbolicMatrix:
    def __init__(self, matrix, column_labels=None, free_terms=None, display_mode="table"):
        self.matrix = Matrix(matrix)
        self.operations = []
        self.display_mode = display_mode

        if column_labels:
            if len(column_labels) != self.matrix.cols:
                raise ValueError("Liczba etykiet kolumn musi być równa liczbie kolumn w macierzy.")
            self.column_labels = column_labels
        else:
            self.column_labels = [f"x{i+1}" for i in range(self.matrix.cols)]

        if free_terms:
            if len(free_terms) != self.matrix.rows:
                raise ValueError("Liczba wyrazów wolnych musi być równa liczbie wierszy w macierzy.")
            self.free_terms = free_terms
        else:
            self.free_terms = [0 for _ in range(self.matrix.rows)]

        display(Markdown("**Początkowe równania:**"))
        self.display_matrix()

    def set_display_mode(self, mode):
        if mode not in ["table", "equations"]:
            raise ValueError("Tryb wyświetlania musi być 'table' lub 'equations'.")
        self.display_mode = mode

    def display_matrix(self):
        if self.display_mode == "table":
            self._display_table()
        elif self.display_mode == "equations":
            self._display_equations()

    def _display_table(self):
        matrix_with_free_terms = self.matrix.row_join(Matrix(self.free_terms))
        headers = self.column_labels + ["Wyraz wolny"]
        display(Markdown(self._matrix_to_markdown(matrix_with_free_terms, headers)))

    def _matrix_to_markdown(self, matrix, headers):
        markdown_str = "| " + " | ".join(headers) + " |\n"
        markdown_str += "| " + " | ".join(["---"] * len(headers)) + " |\n"
        for i in range(matrix.rows):
            row = [str(matrix[i, j]) for j in range(matrix.cols)]
            markdown_str += "| " + " | ".join(row) + " |\n"
        return markdown_str

    def _display_equations(self):
        equations = []
        for i in range(self.matrix.rows):
            terms = []
            for j in range(self.matrix.cols):
                coef = self.matrix[i, j]
                var = symbols(self.column_labels[j])

                # Sprawdzamy, czy współczynnik jest dodatni, ujemny czy zero
                if coef > 0:
                    if j == 0:
                        terms.append(f"{latex(coef)} {latex(var)}")
                    else:
                        terms.append(f"+ {latex(coef)} {latex(var)}")
                elif coef < 0:
                    terms.append(f"- {latex(abs(coef))} {latex(var)}")
                elif coef == 0:
                    if j == 0:
                        terms.append(f"0 {latex(var)}")
                    else:
                        terms.append(f"+0 {latex(var)}")

            lhs = " ".join(terms)
            rhs = latex(self.free_terms[i])

            equation = f"{lhs} = {rhs}"
            equations.append(equation)

        display(Markdown("Układ równań:"))
        display(Markdown(f"$$\\begin{{aligned}} {self._equations_to_latex(equations)} \\end{{aligned}}$$"))

    def _equations_to_latex(self, equations):
        return " \\\\ ".join(equations)

    def add_row(self, target_row, source_row, coefficient):
        target_idx = self._validate_row_number(target_row)
        source_idx = self._validate_row_number(source_row)

        # Operacja na macierzy
        self.matrix.row_op(target_idx, lambda v, j: v + coefficient * self.matrix[source_idx, j])

        # Operacja na wyrazach wolnych
        self.free_terms[target_idx] += coefficient * self.free_terms[source_idx]

        operation_str = f"w{target_row} = w{target_row} + {coefficient}*w{source_row}"
        self.operations.append(operation_str)
        display(Markdown(f"**Operacja:** {operation_str}"))
        self.display_matrix()

    def multiply_row(self, row, coefficient):
        row_idx = self._validate_row_number(row)

        # Operacja na macierzy
        self.matrix.row_op(row_idx, lambda v, _: coefficient * v)

        # Operacja na wyrazach wolnych
        self.free_terms[row_idx] *= coefficient

        operation_str = f"w{row} = {coefficient}*w{row}"
        self.operations.append(operation_str)
        display(Markdown(f"**Operacja:** {operation_str}"))
        self.display_matrix()

    def swap_rows(self, row1, row2):
        row1_idx = self._validate_row_number(row1)
        row2_idx = self._validate_row_number(row2)

        self.matrix.row_swap(row1_idx, row2_idx)
        self.free_terms[row1_idx], self.free_terms[row2_idx] = self.free_terms[row2_idx], self.free_terms[row1_idx]

        operation_str = f"Zamiana w{row1} <-> w{row2}"
        self.operations.append(operation_str)
        display(Markdown(f"**Operacja:** {operation_str}"))
        self.display_matrix()

    def swap_columns(self, col1, col2):
        col1_idx = self._validate_col_number(col1)
        col2_idx = self._validate_col_number(col2)

        self.matrix.col_swap(col1_idx, col2_idx)
        self.column_labels[col1_idx], self.column_labels[col2_idx] = self.column_labels[col2_idx], self.column_labels[col1_idx]

        operation_str = f"Zamiana kol{col1} <-> kol{col2}"
        self.operations.append(operation_str)
        display(Markdown(f"**Operacja:** {operation_str}"))
        self.display_matrix()

    def _validate_row_number(self, row):
        if not isinstance(row, int):
            raise TypeError("Numer wiersza musi być liczbą całkowitą.")
        if row < 1 or row > self.matrix.rows:
            raise IndexError(f"Numer wiersza musi być w zakresie od 1 do {self.matrix.rows}.")
        return row - 1

    def _validate_col_number(self, col):
        if not isinstance(col, int):
            raise TypeError("Numer kolumny musi być liczbą całkowitą.")
        if col < 1 or col > self.matrix.cols:
            raise IndexError(f"Numer kolumny musi być w zakresie od 1 do {self.matrix.cols}.")
        return col - 1

    def print_operations(self):
        display(Markdown("**Wykonane operacje:**"))
        for op in self.operations:
            print(op)

# Przykład użycia:
m = SymbolicMatrix([[1,2,3], [4,5,6], [7,8,12]], column_labels=["x1", "x2", "x3"], free_terms=[10, 11, 18], display_mode="equations")


**Początkowe równania:**

Układ równań:

$$\begin{aligned} 1 x_{1} + 2 x_{2} + 3 x_{3} = 10 \\ 4 x_{1} + 5 x_{2} + 6 x_{3} = 11 \\ 7 x_{1} + 8 x_{2} + 12 x_{3} = 18 \end{aligned}$$

In [ ]:
m.add_row(2, 1, -4)

**Operacja:** w2 = w2 + -4*w1

Układ równań:

$$\begin{aligned} 1 x_{1} + 2 x_{2} + 3 x_{3} = 10 \\ 0 x_{1} - 3 x_{2} - 6 x_{3} = -29 \\ 7 x_{1} + 8 x_{2} + 12 x_{3} = 18 \end{aligned}$$

In [ ]:
m.add_row(3, 1, -7)

**Operacja:** w3 = w3 + -7*w1

Układ równań:

$$\begin{aligned} 1 x_{1} + 2 x_{2} + 3 x_{3} = 10 \\ 0 x_{1} - 3 x_{2} - 6 x_{3} = -29 \\ 0 x_{1} - 6 x_{2} - 9 x_{3} = -52 \end{aligned}$$

In [ ]:
m.add_row(3, 2, -2)

**Operacja:** w3 = w3 + -2*w2

Układ równań:

$$\begin{aligned} 1 x_{1} + 2 x_{2} + 3 x_{3} = 10 \\ 0 x_{1} - 3 x_{2} - 6 x_{3} = -29 \\ 0 x_{1} +0 x_{2} + 3 x_{3} = 6 \end{aligned}$$

In [ ]:
# mamy 3 równania, więc musimy zakończyć działanie
# definicja symboli
x1, x2, x3 = symbols('x1 x2 x3')
# Definicja równań
eq1 = Eq(x1 + 2*x2 + 3*x3, 10)
eq2 = Eq(-3*x2-6*x3,-29)
eq3 = Eq(3*x3, 6)

In [ ]:
sol_x3 = sp.solve(eq3, x3)[0]
sol_x3

2

In [ ]:
# podstawiamy rozwiązanie x3 do równania 2
eq2 = eq2.subs(x3, sol_x3)
# rozwiązujemy równanie 2
sol_x2 = sp.solve(eq2, x2)[0]
sol_x2

17/3

In [ ]:
# podstawiamy rozwiązania x2 i x3 do równania 1
eq1 = eq1.subs({x2: sol_x2, x3: sol_x3})
# rozwiązujemy równanie 1
sol_x1 = sp.solve(eq1, x1)[0]
sol_x1

-22/3

---

## Zadania dla studentów

$$
\left\{\begin{array}{c}{x+2 y-2 z=4} \\ {2 x+y+z=0} \\ {3 x+2 y+z=1}\end{array}\right.\quad
\left\{\begin{array}{c}{x+y+z-t=2} \\ {2 x+y+z=3} \\ {-x+z-t=0} \\ {3 x+2 y-z+2 t=-1}\end{array}\right.\quad
\left\{\begin{array}{c}{x+y-z-t=0} \\ {2 x+3 y-2 z+t=4} \\ {3 x+5 z=0} \\ {-x+y-3 z+2 t=3}\end{array}\right.
$$

In [ ]:
import numpy as np

# Definiujemy macierz współczynników i wektor wyrazów wolnych
A = np.array([
    [1, 2, -2],
    [2, 1, 1],
    [3, 2, 1]
], dtype=float)

B = np.array([4, 0, 1], dtype=float)

# Eliminacja Gaussa
n = len(B)
for i in range(n):
    # Dzielimy wiersz i przez A[i, i] (aby ustalić jedynkę na diagonali)
    A[i] = A[i] / A[i, i]
    B[i] = B[i] / A[i, i]

    # Eliminacja zmiennych poniżej
    for j in range(i + 1, n):
        factor = A[j, i]
        A[j] = A[j] - factor * A[i]
        B[j] = B[j] - factor * B[i]

# Otrzymaliśmy macierz trójkątną, więc wykonujemy podstawienia wsteczne
X = np.zeros(n)
for i in range(n - 1, -1, -1):
    X[i] = B[i] - np.dot(A[i, i + 1:], X[i + 1:])

# Wynik
print("Rozwiązanie układu równań:")
print("x =", X[0])
print("y =", X[1])
print("z =", X[2])


In [ ]:
import numpy as np

# Definiujemy macierz współczynników i wektor wyrazów wolnych
A = np.array([
    [1, 1, 1, -1],
    [2, 1, 1, 0],
    [-1, 0, 1, -1],
    [3, 2, -1, 2]
], dtype=float)

B = np.array([2, 3, 0, -1], dtype=float)

# Eliminacja Gaussa
n = len(B)
for i in range(n):
    # Dzielimy wiersz i przez A[i, i] (aby ustalić jedynkę na diagonali)
    A[i] = A[i] / A[i, i]
    B[i] = B[i] / A[i, i]

    # Eliminacja zmiennych poniżej
    for j in range(i + 1, n):
        factor = A[j, i]
        A[j] = A[j] - factor * A[i]
        B[j] = B[j] - factor * B[i]

# Otrzymaliśmy macierz trójkątną, więc wykonujemy podstawienia wsteczne
X = np.zeros(n)
for i in range(n - 1, -1, -1):
    X[i] = B[i] - np.dot(A[i, i + 1:], X[i + 1:])

# Wynik
print("Rozwiązanie układu równań:")
print("x =", X[0])
print("y =", X[1])
print("z =", X[2])
print("t =", X[3])
